In [2]:
import sys
sys.path.append("../metaheuristics") 
%load_ext autoreload
%autoreload 1
%aimport algorithme_genetique

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from algorithme_genetique import*

In [4]:
vrptw = load_solomon('simple.csv', nb_cust=10, vehicle_speed=100)
instance =init_vrpga(vrptw)
best_solutions={}

In [81]:
for i in range(40):
    instance.evolution()
print(instance.best_solution,instance.fitness(instance.best_solution))

[0, 5, 3, 0, 6, 10, 4, 0, 8, 0, 7, 9, 1, 0, 2, 0] -5620.84314940416


In [82]:
for i in range(10):
    for i in range(400):
        instance.evolution()
    print(instance.best_solution,instance.fitness(instance.best_solution))
    best_solutions[str(instance.best_solution)]=instance.fitness(instance.best_solution)

[0, 5, 7, 3, 6, 4, 10, 1, 0, 8, 9, 2, 0] -2411.0398677072353
[0, 5, 10, 6, 4, 1, 0, 3, 7, 8, 9, 2, 0] -2469.901798976494
[0, 5, 7, 3, 10, 1, 0, 8, 6, 4, 9, 2, 0] -2425.6667036401786
[0, 5, 2, 0, 7, 8, 9, 10, 6, 4, 0, 3, 1, 0] -3476.5467041758675
[0, 7, 8, 9, 0, 3, 6, 4, 2, 10, 1, 0, 5, 0] -3425.4882803104597
[0, 7, 8, 2, 0, 3, 6, 4, 9, 10, 1, 0, 5, 0] -3448.674311569822
[0, 7, 8, 9, 0, 3, 6, 4, 2, 0, 5, 10, 1, 0] -3408.1942041989255
[0, 7, 5, 10, 6, 0, 8, 4, 9, 2, 0, 3, 1, 0] -3545.15106935875
[0, 3, 0, 10, 2, 0, 8, 4, 1, 0, 7, 0, 5, 9, 6, 0] -5611.996581961093
[0, 2, 0, 5, 7, 8, 0, 3, 10, 6, 4, 9, 1, 0] -3477.4161590645153


In [76]:
best_solutions

{'[0, 5, 8, 7, 6, 4, 0, 3, 9, 2, 10, 1, 0]': -2507.0156371393223,
 '[0, 7, 8, 6, 4, 10, 9, 0, 3, 5, 1, 2, 0]': -2493.565753396381,
 '[0, 5, 8, 1, 0, 7, 9, 2, 0, 3, 0, 10, 6, 4, 0]': -4542.028143190721,
 '[0, 5, 10, 3, 6, 4, 0, 2, 1, 0, 9, 0, 8, 7, 0]': -4449.921700970464,
 '[0, 5, 10, 3, 4, 6, 1, 2, 0, 7, 8, 9, 0]': -2383.428459060622}

In [78]:
best_key = max(best_solutions, key=best_solutions.get)
print(best_key)

[0, 5, 10, 3, 4, 6, 1, 2, 0, 7, 8, 9, 0]


Best records:

In [83]:
[0, 5, 3, 7, 8, 9, 10, 6, 4, 2, 1, 0]

[0, 5, 3, 7, 8, 9, 10, 6, 4, 2, 1, 0]

DATA GENERATOR

In [5]:
import random
import numpy as np

In [26]:
def generator_demand(low=8,high=42):
    tmp=int(np.random.normal(20,10))
    while(tmp<low or tmp>high):
        tmp=int(np.random.normal(20,10))
    return tmp

In [34]:
def generator_time_window(right=1300,mid_during=400,f=200):
    start=int(np.random.normal(450,f))
    while(start>1050 or start<0):
        start=int(np.random.normal(450,f))  
    
    during=int(np.random.normal(mid_during,f))
    while(during<200):
        during=int(np.random.normal(mid_during,f))  
    end=min(start+during,right)

    return start,end


In [56]:
def generator():

    return random.randint(0, 100),random.randint(0, 100),generator_demand(),generator_time_window(),90

In [57]:
for i in range(41):
    print((str(generator())).replace(' ','').replace('(','').replace(')',''))

77,10,19,169,662,90
60,97,22,438,922,90
69,14,36,617,1072,90
11,24,9,762,1107,90
79,42,28,541,887,90
44,10,31,908,1300,90
88,99,24,433,998,90
71,9,29,454,893,90
1,31,27,780,1148,90
35,5,30,758,1091,90
66,48,28,528,1207,90
68,97,21,733,973,90
13,2,8,714,1089,90
77,18,23,161,492,90
37,90,13,30,480,90
56,91,13,543,859,90
93,0,23,123,542,90
52,52,35,153,526,90
77,61,23,352,769,90
67,41,37,501,1196,90
18,9,25,313,720,90
82,60,25,124,514,90
69,21,16,398,830,90
19,20,18,330,959,90
85,48,33,434,1115,90
72,76,27,562,888,90
99,16,11,109,753,90
57,65,16,517,1141,90
65,13,22,520,1233,90
64,44,38,391,618,90
80,39,20,493,818,90
63,44,26,110,628,90
11,49,22,304,673,90
83,84,24,625,1092,90
4,97,19,492,1012,90
24,38,26,938,1300,90
25,98,12,516,855,90
1,36,11,706,1300,90
69,100,15,1,260,90
99,82,19,353,781,90
90,78,32,392,1020,90


In [49]:
for i in range(20):
    print(str(i))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [23]:
for _ in range(20):
    print(int(np.random.normal(20,10)))

36
7
16
32
22
1
7
33
21
6
14
23
14
5
16
21
20
8
19
14
